In [17]:
import pandas as pd
import os 
import re
os.chdir('/home/dqw_zyf/GraphDTI/data')

In [18]:
import gensim 
import jieba
from gensim.models import Word2Vec 
import random

ImportError: No module named 'gensim'

In [13]:
alldata = pd.read_csv('alldata.csv')#559009

In [14]:
#构建dataset
alldata = alldata[(alldata['mol'].notna())] #559008
print(len(alldata))

491718


In [15]:
alldata['counts']=alldata['mol'].map(lambda x: x.count('C'))
#去掉无机物

alldata = alldata[alldata.counts > 3 ].reset_index(drop = True)
alldata  = alldata.drop('counts',axis=1) #491718

In [16]:
len(alldata)

491718

In [17]:
vector_size = 100
window=3
negative = 12


def word2vec(vector_size,window,negative,data):
    word_vec = pd.DataFrame()
    dictionary=[]
    Index = []
    #data=read_data()
    
    texts = [[word for word in re.findall(r'.{3}',document)] for document in list(data)]
    model = Word2Vec(texts,vector_size=100,window=3,min_count=1,negative=12,sg=1,sample=0.001,hs=1,workers=4)
    #model = Word2Vec(texts,window=3,min_count=1,negative=12,sg=1,sample=0.001,hs=1,workers=4)
    vectors = pd.DataFrame([model.wv[word] for word in (model.wv.index_to_key)])
    #vectors = pd.DataFrame (model.wv.key_to_index)
    vectors['Word'] = list(model.wv.index_to_key)

    for i in range(len(data)):
        Index.append(i)
    # Word segmentation
    for i in range(len(texts)):
        i_word=[]         
        for w in range(len(texts[i])):
            i_word.append(Index[i])    
        dictionary.extend(i_word)
    word_vec['Id'] = dictionary

    # word vectors generation
    dictionary=[]
    for i in range(len(texts)):
        i_word=[]         
        for w in range(len(texts[i])):
            i_word.append(texts[i][w])    
        dictionary.extend(i_word)
    word_vec['Word'] = dictionary
    del dictionary,i_word
    word_vec = word_vec.merge(vectors,on='Word', how='left')
    #word_vec = word_vec.drop('Word',axis=1)
    word_vec.columns = ['Id']+['word']+["vec_{0}".format(i) for i in range(0,vector_size)]

    return word_vec


In [18]:
def feature_embeddings(dims):
    word_vec = word2vec(dims,vector_size,negative_size)
    word_vec=word_vec.drop('Word',axis=1)
    name = ["vec_{0}".format(i) for i in range(0,dims)]
    feature_embeddings = pd.DataFrame(word_vec.groupby(['Id'])[name].agg('mean')).reset_index()
    feature_embeddings.columns=["Index"]+["mean_ci_{0}".format(i) for i in range(0,dims)]
    return feature_embeddings

In [19]:
#选取ChEMBL+BingdingDB+PubChem数据库作为语料库
#corpus1_mol  = alldata[alldata['Curation/DataSource']=='ChEMBL']['mol'].unique()   #244040
#corpus1_tar  = alldata[alldata['Curation/DataSource']=='ChEMBL']['BindingDB Target Chain Sequence'].unique() #4532
corpus2_mol  = alldata['mol'].unique()   #244040
corpus2_tar  = alldata['BindingDB Target Chain Sequence'].unique() #4532


In [22]:
tar_vec = word2vec(vector_size,window,negative,data = corpus2_tar)

#tar_vec=tar_vec.drop('word',axis=1)
name = ["vec_{0}".format(i) for i in range(0,vector_size)]
tar_embeddings = pd.DataFrame(tar_vec.groupby(['Id'])[name].agg('mean')).reset_index()
tar_embeddings.columns=["tar_id"]+["mean_tar_{0}".format(i) for i in range(0,vector_size)]

In [23]:
#tar_embeddings  = tar_embeddings.drop("Index",axis=1)
tar_embeddings['BindingDB Target Chain Sequence'] = corpus2_tar

In [24]:
tar_embeddings.tail()

,tar_id,mean_tar_0,mean_tar_1,mean_tar_2,mean_tar_3,mean_tar_4,mean_tar_5,mean_tar_6,mean_tar_7,mean_tar_8,...,mean_tar_91,mean_tar_92,mean_tar_93,mean_tar_94,mean_tar_95,mean_tar_96,mean_tar_97,mean_tar_98,mean_tar_99,BindingDB Target Chain Sequence
5094,5094,-0.012719,0.275318,0.092908,-0.106578,0.032645,0.096156,0.029102,0.077260,-0.104480,...,0.213128,0.086025,0.342312,-0.077174,-0.092317,-0.221209,-0.150823,0.015575,0.004524,MIRLGAPQTLVLLTLLVAAVLRCQGQDVQEAGSCVQDGQRYNDKDV...
5095,5095,-0.038878,0.222541,0.135838,-0.019024,0.116351,0.153735,0.044668,-0.051825,-0.122250,...,0.170908,0.103118,0.359449,-0.083759,0.055356,-0.149844,-0.135874,-0.106971,0.031105,MALLHSGRVLPGIAAAFHPGLAAAASARASSWWTHVEMGPPDPILG...
5096,5096,-0.034275,0.239435,0.130043,-0.040077,0.119818,0.223857,0.004845,-0.080334,-0.108943,...,0.139703,0.115951,0.329525,-0.080442,0.073521,-0.125632,-0.131919,-0.098848,0.035441,MNFIKDNSRALIQRMGMTVIKQITDDLFVWNVLNREEVNIICCEKV...
5097,5097,-0.067647,0.225892,0.120734,-0.049174,0.129922,0.232445,-0.003036,-0.055368,-0.133887,...,0.141833,0.101546,0.321413,-0.068654,0.077306,-0.140073,-0.104305,-0.100214,0.031025,MSARRQELQDRAIVKIAAHLPDLIVYGDFSPERPSVKCFDGVLMFV...
5098,5098,-0.049665,0.173407,0.108855,-0.070041,0.072173,0.171251,0.093606,-0.042198,-0.136079,...,0.183476,0.158116,0.363481,-0.092311,0.126279,-0.140826,-0.133228,-0.098072,-0.017329,MKVSAALLCLLLIAATFIPQGLAQPDAINAPVTCCYNFTNRKISVQ...


In [25]:
mol_vec = word2vec(vector_size,window,negative,data = corpus2_mol)
name = ["vec_{0}".format(i) for i in range(0,vector_size)]
mol_embeddings = pd.DataFrame(mol_vec.groupby(['Id'])[name].agg('mean')).reset_index()
mol_embeddings.columns=["mol_id"]+["mean_mol_{0}".format(i) for i in range(0,vector_size)]
mol_embeddings['mol'] = corpus2_mol
#mol_embeddings  = mol_embeddings.drop("mol_id",axis=1)

NameError: name 'corpus1_mol' is not defined

In [26]:
mol_embeddings['mol'] = corpus2_mol

In [27]:
#tar_embeddings.to_csv('corpus1_tar_embeddings.csv',index =False)
#mol_embeddings.to_csv('corpus1_mol_embeddings.csv',index = False)
tar_embeddings.to_csv('corpus2_tar_embeddings.csv',index =False)
mol_embeddings.to_csv('corpus2_mol_embeddings.csv',index = False)


In [28]:
alldata.to_csv('alldata.csv',index = False)

In [29]:
mol_embeddings.tail()

,mol_id,mean_mol_0,mean_mol_1,mean_mol_2,mean_mol_3,mean_mol_4,mean_mol_5,mean_mol_6,mean_mol_7,mean_mol_8,...,mean_mol_91,mean_mol_92,mean_mol_93,mean_mol_94,mean_mol_95,mean_mol_96,mean_mol_97,mean_mol_98,mean_mol_99,mol
281871,281871,0.041941,0.175699,0.059366,-0.004086,-0.037789,-0.021656,-0.058616,-0.006336,0.078971,...,0.073406,0.014812,0.013325,0.116843,0.017214,0.196944,-0.054805,0.049229,-0.080450,O([C@H]1[C@H](O)[C@@H](O)[C@H](O)[C@@H](CO)O1)...
281872,281872,-0.007227,0.149411,-0.043619,-0.170785,-0.082687,0.039564,-0.037594,-0.013339,0.118947,...,-0.044760,-0.004789,-0.085263,0.086711,-0.047406,0.220045,-0.050671,0.067193,-0.165913,O(C[C@@H]1[C@@H](O)[C@H](O)[C@@H](O)[C@H](Oc2c...
281873,281873,0.022933,0.169662,-0.023590,-0.144657,-0.104247,0.001381,-0.053712,-0.041902,0.125689,...,0.060111,0.008148,-0.083598,0.096347,-0.036316,0.224573,-0.021006,0.092585,-0.132709,O(C[C@@H]1[C@@H](O)[C@H](O)[C@@H](O)[C@H](Oc2c...
281874,281874,0.029901,0.164326,0.051684,-0.013896,-0.017539,-0.004060,-0.074515,0.018421,0.063137,...,0.043103,0.034345,0.031916,0.112153,-0.004132,0.223699,-0.047207,0.091703,-0.103646,O(Cc1c(O)c2C(=O)c3c(C(=O)c2cc1O)cccc3)[C@H]1[C...
281875,281875,-0.039832,0.126128,0.043430,-0.154752,-0.004561,0.023689,-0.049957,0.086389,0.092846,...,-0.063494,0.013401,-0.004960,0.069559,-0.012962,0.265099,-0.136502,0.073263,-0.133685,O(C)c1cc2C(=O)c3c(c(O)c(CO)c(O[C@H]4[C@H](O)[C...


In [ ]:
#NS5
'EFGKAKGSRAIWYMWLGARFLEFEALGFLNEDHWMGRENSGGGVEGLGLQRLGYVLEEMSRIPGGRMYADDTAGWDTRISRFDLENEALITNQMEKGHRALALAIIKYTYQNKV\n
 VKVLRPAEKGKTVMDIISRQDQRGSGQVVTYALNTFTNLVVQLIRNMEAEEVLEMQDLWLLRRSEKVTNWLQSNGWDRLKRMAVSGDDCVVKPIDDRFAHALRFLNDMG\n
   KVRKDTQEWKPSTGWDNWEEVPFCSHHFNKLHLKDGRSIVVPCRHQDELIGRARVSPGAGWSIRETACLAKSYAQMWQLLYFHRRDLRLMANAICSSVPVDWVPTGRTTWSIHGKGE'

In [4]:
tar_embeddings = pd.read_csv('corpus2_tar_embeddings.csv')

In [5]:
tar_embeddings.head()

,tar_id,mean_tar_0,mean_tar_1,mean_tar_2,mean_tar_3,mean_tar_4,mean_tar_5,mean_tar_6,mean_tar_7,mean_tar_8,...,mean_tar_91,mean_tar_92,mean_tar_93,mean_tar_94,mean_tar_95,mean_tar_96,mean_tar_97,mean_tar_98,mean_tar_99,BindingDB Target Chain Sequence
0,0,0.028382,0.258349,0.084484,-0.047875,0.129138,0.124458,0.009141,-0.022629,-0.098500,...,0.190733,0.132939,0.380325,-0.109288,0.105654,-0.217573,-0.108054,-0.099446,0.019375,PQITLWQRPL VTIKIGGQLK EALLDTGADD TVLEEMNLPG RW...
1,1,0.029465,0.262882,0.080848,-0.052415,0.142362,0.111432,0.005006,-0.023507,-0.090406,...,0.186500,0.131415,0.377519,-0.114872,0.113332,-0.207250,-0.119483,-0.105467,0.023274,PQITLWQRPL VTIKIGGQLK EALLDTGADD TVLEEMSLPG RW...
2,2,-0.020833,0.211167,0.068523,-0.020104,0.086222,0.200455,-0.000080,-0.137738,-0.027405,...,0.078893,0.084825,0.258192,-0.072125,0.062576,-0.157971,-0.082105,-0.073030,-0.056679,PQFSLWKRPV VTAYIEGQPV EVLLDTGADD SIVAGIELGN NY...
3,3,-0.059700,0.275516,0.189792,-0.019265,0.114139,0.230459,-0.004973,-0.029325,-0.079160,...,0.119733,0.012204,0.374560,-0.124122,0.015532,-0.178388,-0.150244,-0.133766,-0.058527,PISPIETVPV KLKPGMDGPK VKQWPLTEEK IKALVEICTE ME...
4,4,-0.082903,0.226635,0.105320,-0.111637,0.138109,0.185506,-0.007795,0.017392,-0.087893,...,0.150620,0.116129,0.405553,-0.051197,0.018855,-0.186015,-0.175932,-0.131321,0.020372,PQVTLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...


In [6]:
len(tar_embeddings)

5099

In [15]:
for i,seq in zip(list(tar_embeddings['tar_id']),list(tar_embeddings['BindingDB Target Chain Sequence'])):
    if 'EFGKAKGSRAIWYMWLGARFLEFEALGFLNEDHWMGRENSGGGVEGLGLQRLGYVLEEMS' in seq:
        print(i)
        print(seq)
        

In [ ]:
query_seq = 'EFGKAKGSRA IWYMWLGARF LEFEALGFLN EDHWMGRENS GGGVEGLGLQ RLGYVLEEMS \n
             RIPGGRMYAD DTAGWDTRIS RFDLENEALI TNQMEKGHRA LALAIIKYTY QNKVVKVLRP \n
     AEKGKTVMDI ISRQDQRGSG QVVTYALNTF TNLVVQLIRN MEAEEVLEMQ DLWLLRRSEK \n
     VTNWLQSNGW DRLKRMAVSG DDCVVKPIDD RFAHALRFLN DMGKVRKDTQ EWKPSTGWDN \n
     WEEVPFCSHH FNKLHLKDGR SIVVPCRHQD ELIGRARVSP GAGWSIRETA CLAKSYAQMW \n
     QLLYFHRRDL RLMANAICSS VPVDWVPTGR TTWSIHGKGE'